# Customizing the Target Parameters of the Resource Estimator 

In [ ]:
import qsharp

In [2]:
%%qsharp
namespace Sample {

    // open Microsoft.Quantum.Intrinsic;
    // open Microsoft.Quantum.Canon;

    operation RandomBit() : Result[] {
        use qubits = Qubit[3];

        // Apply gates
        H(qubits[0]);
        H(qubits[1]);
        T(qubits[2]);

        CCNOT(qubits[0], qubits[1], qubits[2]);

        // Measure the qubits
        let results = [M(qubits[0]), M(qubits[1]), M(qubits[2])];

        // Reset the qubits
        ResetAll(qubits);
        return results;
    }
}

Usually, the resource estimator takes a set of inputs with predefined values:
- Physical qubit model
- QEC code
- Error budget
- Constraints
- Distillation units
- Pareto frontier estimation

However, we can also pre-calculate estimates for an operation and incorporate them (which will come later).

#### Physical Qubit Parameters
For the physcial qubit (gate-based) instruction set, there are four predefined qubit models. These are listed [here](https://learn.microsoft.com/en-us/azure/quantum/overview-resources-estimator), along with the complete predefined qubit parameters. For example, below, we can do a Pareto frontier estimation for two "transmon qubits" (`qubit_gate_ns_e3` operation times and fidelities correspond to "realistic" future transmon qubits) in a surface code, but modify the one qubit gate error rate for each of them.

In [ ]:
from qsharp_widgets import EstimatesOverview

result = qsharp.estimate(
    "Sample.RandomBit()",
    [
        {
        "qubitParams": { "name": "qubit_gate_ns_e3" ,
                        "oneQubitGateErrorRate": 1e-3,
                        },
        "qecScheme": { "name": "surface_code" },
        "estimateType": "frontier", # Pareto frontier estimation
        },
        {
        "qubitParams": { "name": "qubit_gate_ns_e3" ,
                        "oneQubitGateErrorRate": 5e-3,  # Fixed key
                        },
        "qecScheme": { "name": "surface_code" },
        "estimateType": "frontier", # Pareto frontier estimation
        },
    ]
)

EstimatesOverview(result, colors=["#1f77b4","#33ffe6"], runNames=["oneQubitGateErrorRate = 1e-3", "oneQubitGateErrorRate = 5e-3"])

#### Different QEC Schemes
The resource estimator uses the following model to compute logical error rates (note that this can be modified like [here](https://github.com/Alice-Bob-SW/qsharp-alice-bob-resource-estimator), as we will later see)
$$
P = a(\frac{p}{p*})^{\frac{d+1}{2}}
$$
where $a$ is the corssing pre-factor (extracted numerically for simulations), $d$ is the code distance, $p$ is the physical error rate and $p*$ is the QEC threshold. The physical error rate $p$ is extracted from the qubit parameters as the worst-case error rate of any physical Clifford operation performed in the device. For transmons, the only predefined QEC code is the surface code, with the predefined parameters shown [here](https://learn.microsoft.com/en-us/azure/quantum/overview-resources-estimator). 

We can compare two surface code architechtures with differing thresholds.

In [ ]:
result = qsharp.estimate(
    "Sample.RandomBit()",
    [
        {
        "qubitParams": { "name": "qubit_gate_ns_e3" ,
                        },
        "qecScheme": { "name": "surface_code" ,
                      "errorCorrectionThreshold": 0.02},
        "estimateType": "frontier", # Pareto frontier estimation
        },
        {
        "qubitParams": { "name": "qubit_gate_ns_e3" ,
                        },
        "qecScheme": { "name": "surface_code" },
        "estimateType": "frontier", # Pareto frontier estimation
        },
    ]
)

EstimatesOverview(result, colors=["#1f77b4","#33ffe6"], runNames=["Threshold = 0.02", "Threshold = 0.01"])

The  followiing template can be used for different QEC codes:
```
qsharp.estimate("RunProgram()", params=
                {"qecScheme": {
                        "crossingPrefactor": <double>,
                        "errorCorrectionThreshold": <double>,
                        "logicalCycleTime": <formula string>,
                        "physicalQubitsPerLogicalQubit": <formula string>
                    }
                })
```

Further described in the [tutorial](https://learn.microsoft.com/en-us/azure/quantum/overview-resources-estimator), we can modify the error budget, use the `constraints` class to aply constraints on the T factory componenet level and provide specifications for T factories distillation algorithms with the `DistillationUnitSpecification` class.